# Preparing for JDC uploads

1. writing core measures to Stata and SPSS files
2. compress
3. map to sheepdog
4. upload both sheepdog and core measure files

In [1]:
import json
from pathlib import Path
from jdc_utils.utils import zip_package
from jdc_utils.encoding import core_measures as encodings
from dataforge.frictionless.categoricals import table_encode
from frictionless import Package,transform

In [2]:
jdc_utils_pwd = Path().resolve().parent

## Write package

## Write Stata and SPSS

In [3]:
os.chdir(jdc_utils_pwd)
datapackage_paths = [
    path.resolve() 
    for path in Path().glob("../*/*/core-measures-*/") 
    if path.is_dir()
]

In [4]:
datapackage_paths

[WindowsPath('C:/Users/kranz-michael/projects/jcoin-chestnut/tmp/core-measures-chestnut'),
 WindowsPath('C:/Users/kranz-michael/projects/jcoin-nyu/tmp/core-measures-nyu'),
 WindowsPath('C:/Users/kranz-michael/projects/jcoin-uky/tmp/core-measures-uky'),
 WindowsPath('C:/Users/kranz-michael/projects/jcoin-yale-hiv/tmp/core-measures-yale_hiv')]

In [5]:
os.chdir(jdc_utils_pwd)

for path in datapackage_paths:
    os.chdir(path)
    report_path = Path('report.json')
    if report_path.exists():
        report = json.loads(report_path.read_text()) #this is a cherry picked version just to indicate whether datapackage is valid
        package = Package('data-package.json')
    else:
        report = {'valid':False}
    if report['valid']:
        print(package['name'])
        #write stata and spss
        for source in package['resources']:
            print(f"-->{source['name']}")
            source_path = Path(source['path'])
            target_spss = transform(source,steps=[table_encode(encodings.fields,encodings.reserve['spss'])])
            target_stata = transform(source,steps=[table_encode(encodings.fields,encodings.reserve['stata'])])
            target_spss_params = target_spss.write(source_path.with_suffix(".sav"))
            target_stata_params = target_stata.write(source_path.with_suffix(".dta"))
    else:
        print(f"{path.stem} is not valid or report.json not generated so skipping stata/spss generation")

core-measures-chestnut
-->baseline
-->time-points
core-measures-nyu
-->baseline
-->time-points
core-measures-uky
-->baseline
-->time-points
core-measures-yale_hiv
-->baseline
-->time-points


In [9]:
#compress to directory
for path in datapackage_paths:
    zip_path = jdc_utils_pwd/'tmp'
    zip_path.mkdir(exist_ok=True)
    zip_package(path,jdc_utils_pwd/'tmp')